# CMI BFRB Detection - ベースライン提出ノートブック

このノートブックは、学習済みのベースラインモデルを使用してKaggleコンペティションに提出するためのものです。

In [ ]:
import os
import pandas as pd
import polars as pl
import numpy as np
import joblib
from pathlib import Path
import json

import kaggle_evaluation.cmi_inference_server

In [ ]:
# モデルとスケーラーの読み込み
MODEL_PATH = Path('/kaggle/input/cmi-baseline-model')

# Kaggle環境では入力データセットとして準備されていることを想定
try:
    model = joblib.load(MODEL_PATH / 'baseline_model.pkl')
    scaler = joblib.load(MODEL_PATH / 'baseline_scaler.pkl')
    
    with open(MODEL_PATH / 'baseline_features.json', 'r') as f:
        feature_names = json.load(f)
    
    print("モデルとスケーラーの読み込み完了")
except FileNotFoundError:
    print("警告: モデルファイルが見つかりません。ローカル実行用のパスを使用します。")
    # ローカル実行時のパス
    local_model_path = Path('/home/chinchilla/kaggle/pjt/cmi-detect-behavior-with-sensor-data/models/trained')
    model = joblib.load(local_model_path / 'baseline_model.pkl')
    scaler = joblib.load(local_model_path / 'baseline_scaler.pkl')
    
    with open(local_model_path / 'baseline_features.json', 'r') as f:
        feature_names = json.load(f)
    
    print("ローカルモデルの読み込み完了")

In [ ]:
def create_features(sensor_data):
    """センサーデータから特徴量を生成"""
    
    # DataFrameの型を確認・変換
    if isinstance(sensor_data, pl.DataFrame):
        sensor_data = sensor_data.to_pandas()
    
    # セッション単位で特徴量を作成
    features = []
    
    for session_id in sensor_data['session_id'].unique():
        session_data = sensor_data[sensor_data['session_id'] == session_id]
        
        feature_dict = {'session_id': session_id}
        
        # 各センサーの統計的特徴量
        sensor_cols = ['accel_x', 'accel_y', 'accel_z', 'gyro_x', 'gyro_y', 'gyro_z', 
                      'magnet_x', 'magnet_y', 'magnet_z', 'proximity', 'temperature', 'signal_strength']
        
        for col in sensor_cols:
            if col in session_data.columns:
                # 基本統計量
                feature_dict[f'{col}_mean'] = session_data[col].mean()
                feature_dict[f'{col}_std'] = session_data[col].std()
                feature_dict[f'{col}_min'] = session_data[col].min()
                feature_dict[f'{col}_max'] = session_data[col].max()
                feature_dict[f'{col}_median'] = session_data[col].median()
                
                # パーセンタイル
                feature_dict[f'{col}_q25'] = session_data[col].quantile(0.25)
                feature_dict[f'{col}_q75'] = session_data[col].quantile(0.75)
                
                # 範囲
                feature_dict[f'{col}_range'] = session_data[col].max() - session_data[col].min()
                
                # 偏度・尖度
                feature_dict[f'{col}_skew'] = session_data[col].skew()
                feature_dict[f'{col}_kurtosis'] = session_data[col].kurtosis()
        
        # 複合特徴量
        if all(col in session_data.columns for col in ['accel_x', 'accel_y', 'accel_z']):
            # 加速度の合成ベクトル
            accel_magnitude = np.sqrt(session_data['accel_x']**2 + 
                                    session_data['accel_y']**2 + 
                                    session_data['accel_z']**2)
            feature_dict['accel_magnitude_mean'] = accel_magnitude.mean()
            feature_dict['accel_magnitude_std'] = accel_magnitude.std()
            feature_dict['accel_magnitude_max'] = accel_magnitude.max()
        
        if all(col in session_data.columns for col in ['gyro_x', 'gyro_y', 'gyro_z']):
            # ジャイロの合成ベクトル
            gyro_magnitude = np.sqrt(session_data['gyro_x']**2 + 
                                   session_data['gyro_y']**2 + 
                                   session_data['gyro_z']**2)
            feature_dict['gyro_magnitude_mean'] = gyro_magnitude.mean()
            feature_dict['gyro_magnitude_std'] = gyro_magnitude.std()
            feature_dict['gyro_magnitude_max'] = gyro_magnitude.max()
        
        # 時系列特徴量
        feature_dict['session_length'] = len(session_data)
        feature_dict['sampling_rate'] = len(session_data) / (session_data['timestamp'].max() - session_data['timestamp'].min() + 1e-6)
        
        features.append(feature_dict)
    
    features_df = pd.DataFrame(features)
    
    # NaN値を0で埋める
    features_df = features_df.fillna(0)
    
    return features_df

In [ ]:
def predict(sequence: pl.DataFrame, demographics: pl.DataFrame) -> str:
    """
    シーケンスデータから予測を行う関数
    
    Args:
        sequence: センサーデータ（Polars DataFrame）
        demographics: 人口統計データ（Polars DataFrame）
    
    Returns:
        str: 予測結果（'BFRB detected' または 'Text on phone'）
    """
    try:
        # 特徴量を生成
        features_df = create_features(sequence)
        
        # セッションIDを保存
        session_ids = features_df['session_id'].values
        
        # session_idを除外して特徴量行列を作成
        X = features_df.drop('session_id', axis=1)
        
        # 学習時と同じ特徴量順序に合わせる
        # 不足している特徴量は0で埋める
        for feature in feature_names:
            if feature not in X.columns:
                X[feature] = 0
        
        # 余分な特徴量を削除
        X = X[feature_names]
        
        # 特徴量をスケーリング
        X_scaled = scaler.transform(X)
        
        # 予測実行
        predictions = model.predict(X_scaled)
        
        # 予測結果を返す
        # 1つでもBFRBが検出された場合は 'BFRB detected' を返す
        if np.any(predictions == 1):
            return 'BFRB detected'
        else:
            return 'Text on phone'
            
    except Exception as e:
        print(f"予測中にエラーが発生しました: {e}")
        # エラー時はデフォルト予測を返す
        return 'Text on phone'

In [ ]:
# 推論サーバーの設定と実行
inference_server = kaggle_evaluation.cmi_inference_server.CMIInferenceServer(predict)

if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    # 本番実行時
    print("本番環境で推論サーバーを開始します")
    inference_server.serve()
else:
    # ローカル実行時
    print("ローカル環境でテスト実行します")
    inference_server.run_local_gateway(
        data_paths=(
            '/kaggle/input/cmi-detect-behavior-with-sensor-data/test.csv',
            '/kaggle/input/cmi-detect-behavior-with-sensor-data/test_demographics.csv',
        )
    )